__Setup IQS Connection__

In [ ]:
import iqs_jupyter
connector = iqs_jupyter.IQSConnectorWidget() # no authentication support at the moment

In [ ]:
iqs = connector.connect() # create the main IQS client for the given access point

In [ ]:
# one can already invoke operations on the server, e.g.
iqs.repository.update_repository()

__Select and load TWC Revision__

In [ ]:
revision_selector = iqs.jupyter_tools.revision_selector_widget(
    # optional: setting default widget state
    initial_workspace = '4d6ce495-1273-452c-a548-36fcd922184e',
    initial_resource  = '34cc77c8-d3ef-40a6-9b91-65786117fe67',
    initial_branch    = 'bd03a239-7836-4d4c-9bcb-eba73b001b1e',
    initial_revision  = 1
)

In [ ]:
iqs.persistent_index.index_revision(revision_selector.value())

In [ ]:
iqs.in_memory_index.load_revision(revision_selector.value())

__Register and Execute Query__

In [ ]:
iqs.queries.unregister_all_queries()

In [ ]:
iqs.queries.register_queries_plain_text('''
pattern dependencies(source : NamedElement, target : NamedElement) {
  Dependency.supplier(dependency, target); Dependency.client(dependency, source);
}
''', query_package="iqs4twc.queries.example")

In [ ]:
qResults = iqs.query_execution.execute_query({
  "revisionDescriptor": revision_selector.value(),
  "queryFQN": "iqs4twc.queries.example.dependencies"
})

__Process query results__

In [ ]:
qResults

In [ ]:
qResults.to_list_of_matches()

In [ ]:
first_result_element = qResults.to_list_of_matches()[0]['source']
first_result_element # check element hover for element descriptor details

In [ ]:
some_element = revision_selector.value().get_element_descriptor_by_id("c1be5a8c-a89e-40aa-87a8-8f7ef9527292") # can also look up directly
some_element

In [ ]:
qResults_restricted = iqs.query_execution.execute_query({
  "revisionDescriptor": revision_selector.value(),
  "queryFQN": "iqs4twc.queries.example.dependencies",
  "parameterBinding": [{"parameter": "target", "value": some_element}]
})
qResults_restricted

__Convert query results to Python/Jupyter-friendly Pandas dataframes and process them as such__

In [ ]:
qResults_df = qResults.to_data_frame()
qResults_df

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
qResults_df.applymap(lambda element: element.element_id).source.value_counts().iplot(
    kind='bar', filename='cufflinks/categorical-bar-chart/frequent_sources',
    yTitle='Number of Dependencies', title='Dependencies from Source',
)

__Connect to OSMC and browse element details__

In [ ]:
osmc_connector = iqs_jupyter.OSMCConnectorWidget()

In [ ]:
osmc = osmc_connector.connect()

In [ ]:
element_info = osmc.show_element_info_widget(first_result_element)
element_info.display()

In [ ]:
element_info.selected_element()

__Sandbox__